<a href="https://colab.research.google.com/github/kimhalyn/OPEN-API/blob/main/hourwal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 라이브러리
# colab에 xmltodict 라이브러리가 기본 설치되지 않으므로 별도 설치(외부 라이브러리)
!pip install xmltodict 
from urllib.request import urlopen, Request
from urllib.parse import urlencode, unquote, quote_plus
from bs4 import BeautifulSoup
import pandas as pd
import requests, xmltodict, json
from xml.etree.ElementTree import Element, SubElement, ElementTree
import xml.etree.ElementTree as ET
import pickle
import numpy as np

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import requests, bs4
import pandas as pd
from lxml import html
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from time import sleep
import csv

url = 'http://opendata.kwater.or.kr/openapi-data/service/pubd/dam/excllncobsrvt/hourwal/list' #!!!url 주의
key = 'tMF2MGGUmsNNPf15Nr91+UA4YVQiWXVjLS7R0odxGSR0yfNjcHFkdRpCT6hXbILlBP1T+TnDJYz1kkDjiaIiQg=='
 
api_key = requests.utils.unquote(key) #디코딩

# 댐코드 및 수위 관측소 코드 불러오기
data1 = pd.read_csv('/content/drive/MyDrive/water_resource_csv/walcode.csv') 
data2 = pd.read_csv('/content/drive/MyDrive/water_resource_csv/walcode.csv') 

# 댐코드 및 수위 관측소 코드 조회(!!!!코드 열 지정 주의하기)
damcode = data1.iloc[0:5, 1] 
walcode = data2.iloc[0:5, 3]
xmlrows = []


for code1 in damcode:
  for code2 in walcode:

    Params = '?' + urlencode({quote_plus('ServiceKey') : api_key,
                                  quote_plus('sdate') : "2020-01-01",
                                  quote_plus('stime') : "00",
                                  quote_plus('edate') : "2020-01-01",
                                  quote_plus('etime') : "24",
                                  quote_plus('damcode') : code1, #댐코드
                                  quote_plus('wal') : code2, #수위 관측소 코드
                                  quote_plus('numOfRows') : "800000",
                                  quote_plus('pageNo') : "1"})
    
#  print(Params)
  
  res = requests.get(url + Params).text.encode('utf-8') # 요청에 대한 응답내용(.encode빼기도 함)
  sleep(0.5)
  print(res) 
  
  xmlobj = bs4.BeautifulSoup(res, 'lxml') #lxml 파서(구문분석)  'lxml-xml'
  print(xmlobj)
  
  #item 태그 분리
  rows = xmlobj.findAll('item') # 매치되는 문자열을 리스트로 리턴
  xmlrows.append(rows)

print (xmlrows)


b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><flux>0</flux><hourwal>0.04</hourwal><no>24</no><obsrdt>2020-01-01 01:00</obsrdt></item><item><flux>0</flux><hourwal>0.04</hourwal><no>23</no><obsrdt>2020-01-01 02:00</obsrdt></item><item><flux>0</flux><hourwal>0.04</hourwal><no>22</no><obsrdt>2020-01-01 03:00</obsrdt></item><item><flux>0</flux><hourwal>0.04</hourwal><no>21</no><obsrdt>2020-01-01 04:00</obsrdt></item><item><flux>0</flux><hourwal>0.2</hourwal><no>20</no><obsrdt>2020-01-01 05:00</obsrdt></item><item><flux>0</flux><hourwal>0.21</hourwal><no>19</no><obsrdt>2020-01-01 06:00</obsrdt></item><item><flux>0</flux><hourwal>0.21</hourwal><no>18</no><obsrdt>2020-01-01 07:00</obsrdt></item><item><flux>0</flux><hourwal>0.21</hourwal><no>17</no><obsrdt>2020-01-01 08:00</obsrdt></item><item><flux>0</flux><hourwal>0.23</hourwal><no>16</no><obsrdt>2020-01-01 09:00</obsrdt>

In [ ]:
print(xmlrows)

[[<item><flux>0</flux><hourwal>0.04</hourwal><no>24</no><obsrdt>2020-01-01 01:00</obsrdt></item>, <item><flux>0</flux><hourwal>0.04</hourwal><no>23</no><obsrdt>2020-01-01 02:00</obsrdt></item>, <item><flux>0</flux><hourwal>0.04</hourwal><no>22</no><obsrdt>2020-01-01 03:00</obsrdt></item>, <item><flux>0</flux><hourwal>0.04</hourwal><no>21</no><obsrdt>2020-01-01 04:00</obsrdt></item>, <item><flux>0</flux><hourwal>0.2</hourwal><no>20</no><obsrdt>2020-01-01 05:00</obsrdt></item>, <item><flux>0</flux><hourwal>0.21</hourwal><no>19</no><obsrdt>2020-01-01 06:00</obsrdt></item>, <item><flux>0</flux><hourwal>0.21</hourwal><no>18</no><obsrdt>2020-01-01 07:00</obsrdt></item>, <item><flux>0</flux><hourwal>0.21</hourwal><no>17</no><obsrdt>2020-01-01 08:00</obsrdt></item>, <item><flux>0</flux><hourwal>0.23</hourwal><no>16</no><obsrdt>2020-01-01 09:00</obsrdt></item>, <item><flux>0</flux><hourwal>0.23</hourwal><no>15</no><obsrdt>2020-01-01 10:00</obsrdt></item>, <item><flux>0</flux><hourwal>0.22</hour

In [ ]:

filePath = '/content/drive/MyDrive/water_resource_csv/xmlrows.xml'


with open(filePath, 'wb') as lf:
    pickle.dump(xmlrows, lf)


with open(filePath, 'rb') as lf:
    readList = pickle.load(lf)


In [6]:
# 컬럼 생성
rowList = []
nameList = []
columnList = []
xmlrowsLen = len(xmlrows)
target = 0
for i in range(0, xmlrowsLen):
    
   
    if len(xmlrows[i]) == 0:
       target = target + 1
      
    else:
        break 
columns = xmlrows[target][0].find_all()
columnsLen = len(columns)
for i in range(0, columnsLen):
    nameList.append(columns[i].name)
nameList = ["Damcode"] + ["walobsrvtcode"] + nameList

In [ ]:
xmlrows[0]

[<item><flux>0</flux><hourwal>0.04</hourwal><no>24</no><obsrdt>2020-01-01 01:00</obsrdt></item>,
 <item><flux>0</flux><hourwal>0.04</hourwal><no>23</no><obsrdt>2020-01-01 02:00</obsrdt></item>,
 <item><flux>0</flux><hourwal>0.04</hourwal><no>22</no><obsrdt>2020-01-01 03:00</obsrdt></item>,
 <item><flux>0</flux><hourwal>0.04</hourwal><no>21</no><obsrdt>2020-01-01 04:00</obsrdt></item>,
 <item><flux>0</flux><hourwal>0.2</hourwal><no>20</no><obsrdt>2020-01-01 05:00</obsrdt></item>,
 <item><flux>0</flux><hourwal>0.21</hourwal><no>19</no><obsrdt>2020-01-01 06:00</obsrdt></item>,
 <item><flux>0</flux><hourwal>0.21</hourwal><no>18</no><obsrdt>2020-01-01 07:00</obsrdt></item>,
 <item><flux>0</flux><hourwal>0.21</hourwal><no>17</no><obsrdt>2020-01-01 08:00</obsrdt></item>,
 <item><flux>0</flux><hourwal>0.23</hourwal><no>16</no><obsrdt>2020-01-01 09:00</obsrdt></item>,
 <item><flux>0</flux><hourwal>0.23</hourwal><no>15</no><obsrdt>2020-01-01 10:00</obsrdt></item>,
 <item><flux>0</flux><hourwal>0

In [7]:
print(nameList)

['Damcode', 'walobsrvtcode', 'flux', 'hourwal', 'no', 'obsrdt']


In [8]:
total = []
damcodeLen = len(damcode)
walcodeLen = len(walcode)


def el_code(a): 
    columnList = []
    eachrows = xmlrows[a]
    eachrowslen = len(eachrows)
    for i in range(0, eachrowslen):
        columns = eachrows[i].find_all()
        columnsLen = len(columns)
        for j in range(0, columnsLen):
            eachColumn = columns[j].text 
            columnList.append(eachColumn)
        columnList = [damcode[a]] + [walcode[a]] + columnList
        rowList.append(columnList)
        columnList = []
    return rowList



for a in range(0, damcodeLen, walcodeLen):
    rowList=el_code(a)
    total.append(rowList)

result = pd.DataFrame(total[0], columns=nameList)

   

In [9]:
print(result)

    Damcode  walobsrvtcode flux hourwal  no            obsrdt
0   1007601        1007601    0    0.04  24  2020-01-01 01:00
1   1007601        1007601    0    0.04  23  2020-01-01 02:00
2   1007601        1007601    0    0.04  22  2020-01-01 03:00
3   1007601        1007601    0    0.04  21  2020-01-01 04:00
4   1007601        1007601    0     0.2  20  2020-01-01 05:00
5   1007601        1007601    0    0.21  19  2020-01-01 06:00
6   1007601        1007601    0    0.21  18  2020-01-01 07:00
7   1007601        1007601    0    0.21  17  2020-01-01 08:00
8   1007601        1007601    0    0.23  16  2020-01-01 09:00
9   1007601        1007601    0    0.23  15  2020-01-01 10:00
10  1007601        1007601    0    0.22  14  2020-01-01 11:00
11  1007601        1007601    0    0.22  13  2020-01-01 12:00
12  1007601        1007601    0    0.06  12  2020-01-01 13:00
13  1007601        1007601    0    0.06  11  2020-01-01 14:00
14  1007601        1007601    0    0.06  10  2020-01-01 15:00
15  1007

In [10]:
result.to_csv("/content/drive/MyDrive/water_resource_csv/hourwal_test2.csv",encoding="utf-8-sig")

In [ ]:
result